In [164]:
def generate_points(n_points, radius):
    angles = np.random.uniform(0, 2 * np.pi, n_points)  # Random angles
    r = np.sqrt(np.random.uniform(0, 1, n_points)) * radius  # Random radius with correct distribution
    x = r * np.cos(angles)
    y = r * np.sin(angles)
    return np.column_stack((x, y))  # Return as Nx2 array


def points_inside_circle(circle_center, radius, points):
    points = np.array(points)
    h, k = circle_center
    x, y = np.hsplit(points, 2)
    
    # Calculate the squared distance from the point to the circle center
    dist_squared = (x - h) ** 2 + (y - k) ** 2
    radius_squared = radius ** 2

    # Check if the point is inside or on the circle
    result = dist_squared <= radius_squared
    return result.flatten()


def move_point(point, angle, distance):
    # Convert angle to radians
    angle_rad = np.radians(angle)

    # Calculate the movement vector components
    dx = distance * np.cos(angle_rad)
    dy = distance * np.sin(angle_rad)

    # Create a movement vector for all points
    displacement_vector = np.array([dx, dy])

    # Displace the points
    displaced_point = point + displacement_vector

    return displaced_point.astype(np.double)


def displace_points(circle_center, radius, angle, dist, points):  
    points_moved = move_point(points, angle, dist)
    points_out = ~points_inside_circle(circle_center, radius, points_moved)
    points_out_ix = np.arange(points.shape[0])[points_out]
    points_moved_copy = deepcopy(points_moved)
    for p_ix in points_out_ix:
        point = points[p_ix]
        point_moved = points_moved[p_ix]
        inters_vector_line = circle_line_intersection(circle_center, radius, point, point_moved)
        inters_0, inters_1 = inters_vector_line
        inters = [inters_0, inters_1]
        dist_ = [distance.euclidean(i, point_moved) for i in inters]
        new_orig = inters[np.argmax(dist_)]
        left_disp_orig = inters[np.argmin(dist_)]
        left_disp_vec = vector_points(left_disp_orig, point_moved)
        point_moved_new = new_orig + left_disp_vec
        points_moved_copy[p_ix] = point_moved_new
    return points_moved, points_moved_copy, points_out


def radomised_angle_displace_points(circle_center, angles, radius, dist, points):
    new_points = []
    for p_ix, point in enumerate(points):
        angle = angles[p_ix]
        dist = 1
        point_moved = move_point(point, angle, dist)
        if ~points_inside_circle(circle_center, radius, point_moved)[0]:
            inters_vector_line = circle_line_intersection(circle_center, radius, point, point_moved)
            inters_0, inters_1 = inters_vector_line
            inters = [inters_0, inters_1]
            dist_ = [distance.euclidean(i, point_moved) for i in inters]
            new_orig = inters[np.argmax(dist_)]
            left_disp_orig = inters[np.argmin(dist_)]
            left_disp_vec = vector_points(left_disp_orig, point_moved)
            point_moved_new = new_orig + left_disp_vec
            new_points.append(point_moved_new)
        else:
            new_points.append(point_moved)
    return np.array(new_points)

def circle_line_intersection(circle_center, radius, point1, point2):
    h, k = circle_center
    x1, y1 = point1
    x2, y2 = point2

    # Calculate slope (m) and intercept (b) of the line
    if x2 - x1 == 0:  # vertical line case
        return []  # A vertical line will not intersect unless it's also at the circle's horizontal level

    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1

    # Coefficients of the quadratic equation Ax^2 + Bx + C = 0
    A = 1 + m**2
    B = 2 * (m * (b - k) - h)
    C = (h**2 + (b - k)**2 - radius**2)

    # Calculate the discriminant
    D = B**2 - 4 * A * C

    intersection_points = []

    if D >= 0:  # There are intersections
        # Two possible intersection points if D > 0, one if D = 0
        sqrt_D = np.sqrt(D)

        # Compute both x-coordinates
        x_intersect1 = (-B + sqrt_D) / (2 * A)
        y_intersect1 = m * x_intersect1 + b
        intersection_points.append((x_intersect1, y_intersect1))

        if D > 0:  # Only compute the second intersection point if D > 0
            x_intersect2 = (-B - sqrt_D) / (2 * A)
            y_intersect2 = m * x_intersect2 + b
            intersection_points.append((x_intersect2, y_intersect2))

    return np.array(intersection_points).astype(np.double)


def point_inside_circle(circle_center, radius, point):
    h, k = circle_center
    x, y = point
    
    # Calculate the squared distance from the point to the circle center
    dist_squared = (x - h) ** 2 + (y - k) ** 2
    radius_squared = radius ** 2

    # Check if the point is inside or on the circle
    return dist_squared <= radius_squared


def vector_points(point1, point2):
    x1, y1 = point1
    x2, y2 = point2

    # Calculate the vector components
    dx = x2 - x1
    dy = y2 - y1

    return np.array([dx, dy]).astype(np.double)

In [27]:
circle_diameter = 14
radius = circle_diameter/2
n_points = 500
points = generate_points(n_points, radius)

bool_map = np.zeros(points.shape[0]).astype(bool)
bool_map[points.shape[0]//2 :] = True

In [6]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))
circle = plt.Circle(
    (0, 0), radius=circle_diameter/2, edgecolor="black",
    lw=1, facecolor="none"
)

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax.add_patch(circle)

scat = ax.scatter(points[:,0], points[:,1], c="black")

positions = []
for i in range(300):
    pm, points, po_map = displace_points([0,0], radius, 180, 0.05, points)
    positions.append(points)

def animate(i):
    scat.set_offsets(positions[i])
    return (scat,)

ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(positions) - 1, interval=10)
HTML(ani.to_html5_video())

In [170]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))
circle = plt.Circle(
    (0, 0), radius=circle_diameter/2, edgecolor="black",
    lw=1, facecolor="none"
)

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax.add_patch(circle)

scat = ax.scatter(points[:,0], points[:,1], c="black")
copy_points_moved_subset_0 = deepcopy(points), po_map = displace_points([0,0], radius, 180, 0.05, copy_points[bool_map])
    copy_points[bool_map] = points_moved_subset_0
    pm, points_moved_subset_1, po_map = displace_points([0,0], radius, 0, 0.05, copy_points[~bool_map])
    copy_points[~bool_map] = points_moved_subset_1
    # copy_points = deepcopy(copy_points)
    positions.append(

FFwriter = animation.FFMpegWriter(fps=60)
ani.save('animation.mp4', writer = FFwriter)copy_points)

def animate(i):
    scat.set_offsets(positions[i])
    return (scat,)

ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(positions) - 1, interval=10)
plt.tight_layout()
# HTML(ani.to_html5_video())
positions = []
for i in range(300):
    copy_points = deepcopy(copy_points)
    pm, points

/tmp/ipykernel_111056/2596261396.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1, 1, figsize=(10,10))


In [167]:
np.array(radomised_angle_displace_points([0,0], radius, dist, points)).shape

(500, 2)

In [161]:
rand

False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
True
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
Fa

In [148]:
move_point(point_, 0, 1)

array([ 5.45989583, -3.83905703])

In [147]:
point_

array([ 4.45989583, -3.83905703])